# Libraries

In [ ]:
%matplotlib inline
import cv2
import os

import numpy as np
import mediapipe as mp

# Import mediapipe model for finger and pose detection

In [ ]:
mp_holistic = mp.solutions.holistic # holistic model
mp_drawing = mp.solutions.drawing_utils # drawing utilities

# Functions

In [ ]:
def mediapipe_detection(image, model):
    """
    Takes input 'image' and 'model'.
    
    Applies model to unwriteable image and returns the image with the results.
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # image is no longer writeable
    results = model.process(image)                 # make prediction
    image.flags.writeable = True                   # image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_landmarks(image, results):
    """
    Applies mask over image.
    """
    # draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # removed as we are only interested in relative position of face
    # draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) 
    # draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 
    # draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 

In [ ]:
def draw_styled_landmarks(image, results):
    """
    Applies stylised mask over image.
    """
    # # Draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
    #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
    #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
def extract_keypoints(results):
    """
    Takes key points from results and converts into an array.
    """
    if results.pose_landmarks:
        pose = np.array([[po.x, po.y, po.z] for po in results.pose_landmarks.landmark]).flatten()
    else:
        pose = np.zeros(33*3)

    if results.left_hand_landmarks:
        left = np.array([[lh.x, lh.y, lh.z] for lh in results.left_hand_landmarks.landmark]).flatten()
    else:
        left = np.zeros(21*3)

    if results.right_hand_landmarks:
        right = np.array([[rh.x, rh.y, rh.z] for rh in results.right_hand_landmarks.landmark]).flatten()
    else:
        right = np.zeros(21*3)
    
    return np.concatenate([pose, left, right])

In [ ]:
def extract_coordinates(filepath, savepath):
    """
    Takes in the file path of the videos and save path for labels.

    Go through each frame of the video, apply detection model and save key points. 

    Show the masked image for visual confirmation.

    Returns a frame number to count total frames processed.
    """
    cap = cv2.VideoCapture(filepath)
    frame_num = 1
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        # while(frame_num < 31):
        while cap.isOpened():
            ret, frame = cap.read()

            if ret == True:
            
                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)
                keypoints = extract_keypoints(results)

                save_dest = os.path.join(savepath, str(frame_num))

                np.save(save_dest, keypoints)

                frame_num += 1


 
            else:
                break
    
    cap.release()

    return frame_num

# Folder creation

Create folders for
1. Label folders for new actions
2. Sub-label folders

In [ ]:
# base directory
directory_path = '/mnt/d/GitHub/SSLrecognition/train_data/videos'
# current directory
c_dir = os.getcwd()

actions = np.array(sorted([folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))])) # sorted to follow folder arrangement

# Keypoint extraction
Here we extract the keypoints of the data (videos) by looping through each set of videos in each action folder, then writing the corresponding keypoints. 

This will be used later for model training.

In [ ]:
video_count = 0 # required for processing later
for action in actions:
    counter = 0 # to count video/extracted file
    if not os.path.isdir(os.path.join(c_dir, 'labels', action)):
        os.mkdir(os.path.join(c_dir, 'labels', action))
    for video in os.listdir(os.path.join(c_dir, 'videos', action)): # going through each converted video file in the action
        counter += 1
        video_count += 1
        filepath = os.path.join(os.path.join(c_dir, 'videos', action, video))

        # NOTE: for each video, save keypoints in new subfolder
        # create subfolder if it does not exist
        subfolder = str(action) + '_' + str(counter)
        if not os.path.isdir(os.path.join(c_dir, 'labels', action, subfolder)):
            os.mkdir(os.path.join(c_dir, 'labels', action, subfolder))
        # set new savepath
        savepath = os.path.join(c_dir, 'labels', action, subfolder)
        print(f'Currently reading video {video_count}: {filepath}')
        extract_coordinates(filepath, savepath)
